In [ ]:
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import openai

OPENAI_API_KEY = "sk-.."
openai.api_key = OPENAI_API_KEY

In [ ]:
LLM_MODEL = "gpt-3.5-turbo"
OPENAI_TEMPERATURE = float(0.2)

In [ ]:
import tiktoken as tiktoken
import time


def limit_tokens_from_string(string: str, model: str, limit: int) -> str:
    """Limits the string to a number of tokens (estimated)."""

    try:
        encoding = tiktoken.encoding_for_model(model)
    except:
        encoding = tiktoken.encoding_for_model('gpt2')  # Fallback for others.

    encoded = encoding.encode(string)

    return encoding.decode(encoded[:limit])

In [ ]:
def openai_call(
    system_prompt: str,
    prompt: str,
    model: str = LLM_MODEL,
    temperature: float = OPENAI_TEMPERATURE,
    max_tokens: int = 100,
):
    while True:
        try:
            if not model.lower().startswith("gpt-"):
                # Use completion API
                response = openai.Completion.create(
                    engine=model,
                    prompt=prompt,
                    temperature=temperature,
                    max_tokens=max_tokens,
                    top_p=1,
                    frequency_penalty=0,
                    presence_penalty=0,
                )
                return response.choices[0].text.strip()
            else:
                # Use 4000 instead of the real limit (4097) to give a bit of wiggle room for the encoding of roles.
                # TODO: different limits for different models.

                trimmed_prompt = limit_tokens_from_string(prompt, model, 4000 - max_tokens)

                # Use chat completion API
                messages = [{"role": "system" , "content": system_prompt},{"role": "system", "content": trimmed_prompt}]
                response = openai.ChatCompletion.create(
                    model='gpt-3.5-turbo',
                    messages=messages,
                    temperature=temperature,
                    max_tokens=2000,
                    n=1,
                    stop=None,
                )
                return response.choices[0].message.content.strip()
        except openai.error.RateLimitError:
            print(
                "   *** The OpenAI API rate limit has been exceeded. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.Timeout:
            print(
                "   *** OpenAI API timeout occurred. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.APIError:
            print(
                "   *** OpenAI API error occurred. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.APIConnectionError:
            print(
                "   *** OpenAI API connection error occurred. Check your network settings, proxy configuration, SSL certificates, or firewall rules. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.InvalidRequestError:
            print(
                "   *** OpenAI API invalid request. Check the documentation for the specific API method you are calling and make sure you are sending valid and complete parameters. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.ServiceUnavailableError:
            print(
                "   *** OpenAI API service unavailable. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        else:
            break

In [ ]:
import json

json_file_path1 = '/content/drive/My Drive/colab/hj/with_synonym.json'
json_file_path2 = '/content/drive/My Drive/colab/hj/question_synonym.json'

data = []

with open(json_file_path1, "r", encoding="utf-8") as file:
  data = json.load(file)

system_prompt = """
Your task is to find synonyms for the given word and return those words in a single line.
I'll provide you with one word, and you should find and return up to 10 words that have similar meanings to that word.

Constraints:
- The words should be related to the given word.
- Each word should be a single complete noun without spaces.
- There should be no duplicates.
"""

result = []
for idx, obj in enumerate(data):
  for question in obj["question_nouns"]:
    item = {}
    item['id'] = obj['id']
    item['question_nouns'] = question
    try:
        response = openai_call(system_prompt, question, max_tokens=2000)
        print(response.split())
        item["question_nouns_synonym"] = response.split()
        result.append(item)
    except Exception as e:
        result.append(item)

    print(item)

with open(json_file_path2, "w", encoding="utf-8") as json_file:
  json.dump(result, json_file, indent=4, ensure_ascii=False)

print(f' 열의 내용이 JSON 파일({json_file_path2})로 저장되었습니다.')


{'id': '1', 'question': '전립선은 여성에도 있나요? ', 'answer': '아닙니다. 전립선은 남성에게만 있는 생식기관 입니다', 'question_nouns': ['전립선', '여성'], 'answer_nouns': ['전립선', '남성', '생식기관'], 'question_nouns_synonym': ['전립선비대증', '전립선암', '성염', '여성', '전립선염', '성암', '비대증'], 'answer_nouns_synonym': ['전립선비대증', '생식기', '진단', '전립선암', '성기', '남성', '조기', '치료', '전립선염', '생식기관']}
{'id': '2', 'question': '전립선은 남성에만 있나요? ', 'answer': '맞습니다. 전립선은 남성에게만 있는 생식기관 입니다', 'question_nouns': ['전립선', '남성'], 'answer_nouns': ['전립선', '남성', '생식기관'], 'question_nouns_synonym': ['저하', '불임', '전립선비대증', '전립선암', '남성호르몬', '남성', '기능', '치료', '전립선염', '예방', '남성성', '건강'], 'answer_nouns_synonym': ['전립선비대증', '생식기', '진단', '전립선암', '성기', '남성', '조기', '치료', '전립선염', '예방', '생식기관']}
{'id': '3', 'question': '전립선은? ', 'answer': '전립선은 방광 바로 밑, 직장(直腸) 앞쪽에 있는 밤톨만 한 크기의 남성 생식기관으로, 정액의 일부를 만들어내고 정낭과 함께 정액을 저장하는 역할을 합니다. 위로는 방광경부, 즉 방광에서 요도로 이행하는 부위와 인접해 앞쪽의 치골전립선인대에 고정되어 있고, 아래로는 비뇨생식격막에 의해 고정되어 있습니다.', 'question_nouns': ['전립선'], 'answer_nouns': ['전립선', '방광', '바로', '밑', '직장', '앞쪽